In [ ]:
#for installing lightgbm package to jupyter
#import sys
#!{sys.executable} -m pip install lightgbm

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = 150

#for machine learning
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
import lightgbm as lgb



In [ ]:
# Read in data
train = pd.read_csv('../input/train.csv')
train.info()
train.head()

In [ ]:
test = pd.read_csv('../input/test.csv')
test.info()
test.head()

**Train data have 9557 entries, Test data have 23855 entries. Lets take a look at the statistic of the attributes.**

In [ ]:
train.describe()
test.describe()

** If we look carefully, There is an outlier for attribute rez_esc in test data.**

In [ ]:
test.loc[test.loc[:,"rez_esc"]==99,"rez_esc"]
#We can see that there is only one outlier =99, the rest of the test data is okay
#According to answer from kaggle competition host, the value can be safely changed to 5.
#https://www.kaggle.com/c/costa-rican-household-poverty-prediction/discussion/61403
test.loc[test.loc[:,"rez_esc"]==99,"rez_esc"]=5
test.loc[:,"rez_esc"].describe()

**Now we will deal with missing values in both test and train dataset. **

In [ ]:
#we will first check for missing values in the columns
train_na= pd.DataFrame((train.isnull().sum().values),index=train.columns, columns=['isNA']).sort_values(by=['isNA'],ascending=False)
if train_na.loc[train_na.loc[:,'isNA']>0,:].shape[0]>1 :
    train_na.loc[train_na.loc[:,'isNA']> 0,]

else:
    print('no NA in train set')

test_na= pd.DataFrame((test.isnull().sum().values),index=test.columns, columns=['isNA']).sort_values(by=['isNA'],ascending=False)
if train_na.loc[train_na.loc[:,'isNA']>0,:].shape[0]>1 :
    test_na.loc[test_na.loc[:,'isNA']> 0,]
else:
    print('no NA in test set')

**We can see the missing values are largely from **

****    rez_esc: years behind in school:***

****    v18q1: number of tablets household owns***

****    v2a1: monthly rent payment***

****    meaneduc: average years of education for adults***

****    SQBmeaned is the square of meaneduc***

**    rez_esc: years behind in school:**

    Data is only available if age of individual from 7 to 17 years old. 
    We will set 0 to all other null values

In [ ]:
rez_esc_age=train.loc[train['rez_esc'].isnull()==False, 'age']

plt.hist(x=rez_esc_age,)
plt.xticks(np.arange(min(rez_esc_age), max(rez_esc_age)+1, 1.0),rotation = 60),
plt.ylabel('frequence of rez_esc')
plt.xlabel('Age')
plt.title('Non-null rez_esc Frequency according to age')


**    v2a1: monthly rent payment**
    
    this depends on tipovivi2 and tipovivi3, v2a1 is NA if tipovivi2 or tipovivi3 is 0
    tipovivi2 (a true false statement if an individual owns the house and is paying installment). 
    tipovivi3 (a true false statement if an individual is renting the house). 
    We will assume 0 for NA in v2a1

In [ ]:
tipos=[x for x in train if x.startswith('tipo')]
rentNA_status=train.loc[train['v2a1'].isnull(), tipos].sum()
plt.bar(tipos,rentNA_status,align='center')
plt.xticks([0,1,2,3,4],['Owns and Paid off','Owns and Paying', 'Renting','Precarious','Other'],rotation = 60),
plt.ylabel('Frequency')
plt.title("Missing Rental 'v2a1' according to Home Ownership Status")

****    v18q1: number of tablets household owns***

    This depends on v18q (a true false statement if an individual own a tablet). v18q1 is NA if v18q is 0
    We will assume 0 for NA in v18q1

In [ ]:
Tablet_status=train.loc[train['v18q1'].isnull(), 'v18q']
plt.hist(x=Tablet_status)
plt.xticks([0,1,2],['Do not Own a Table','Owns a Tablet'],rotation=60),
plt.ylabel('Frequency missing value on v18q1')
plt.xlabel('Individual Tablet Ownership (v18q)')
plt.title('Missing value on household tablet ownership vs individual tablet ownership')

**    meaneduc: average years of education for adults**
    
    We will replace this with mode
    
**    SQBmeaned is the square of meaneduc**
    
    replace with square of replaced meaneduc

In [ ]:

plt.figure(figsize=(10,5))
plt.hist(x=train['meaneduc'],bins=int(train['meaneduc'].max()))
plt.xticks(np.arange(min(train['meaneduc']), max(train['meaneduc'])+1),rotation=60),
plt.ylabel('Frequency')
plt.xlabel('average years of education for adults (18+)')
plt.title('Histogram for meaneduc')


In [ ]:
train.loc[:,"meaneduc"].mode()
#train: mode for meaneduc is 6 replace NA with 6, replace SQBmeaned NA to 36
train.loc[train.loc[:,"meaneduc"].isnull()==True,"meaneduc"] = 6
train.loc[train.loc[:,"SQBmeaned"].isnull()==True,"SQBmeaned"] = 36

test.loc[:,"meaneduc"].mode()
#test: mode for meaneduc is 6 replace NA with 6, replace SQBmeaned NA to 36
test.loc[test.loc[:,"meaneduc"].isnull()==True,"meaneduc"] = 6
test.loc[test.loc[:,"SQBmeaned"].isnull()==True,"SQBmeaned"] = 36


#Replace all NA values for remaining 3 attributes with 0no
train.loc[train.loc[:,"rez_esc"].isnull()==True,"rez_esc"] = 0
train.loc[train.loc[:,"v18q1"].isnull()==True,"v18q1"] = 0
train.loc[train.loc[:,"v2a1"].isnull()==True,"v2a1"] = 0

test.loc[test.loc[:,"rez_esc"].isnull()==True,"rez_esc"] = 0
test.loc[test.loc[:,"v18q1"].isnull()==True,"v18q1"] = 0
test.loc[test.loc[:,"v2a1"].isnull()==True,"v2a1"] = 0


In [ ]:
#Check for missing values again:
train_na= pd.DataFrame((train.isnull().sum().values),index=train.columns, columns=['isNA']).sort_values(by=['isNA'],ascending=False)
if train_na.loc[train_na.loc[:,'isNA']>0,:].shape[0]>1 :
    train_na.loc[train_na.loc[:,'isNA']> 0,]

else:
    print('no NA in train set')

test_na= pd.DataFrame((test.isnull().sum().values),index=test.columns, columns=['isNA']).sort_values(by=['isNA'],ascending=False)
if train_na.loc[train_na.loc[:,'isNA']>0,:].shape[0]>1 :
    test_na.loc[test_na.loc[:,'isNA']> 0,]
else:
    print('no NA in test set')

In [ ]:
# Investigate if all individuals in the household have the same poverty target
target_Discrepancy=(train.groupby('idhogar')['Target'].nunique()>1)

print('There are ',target_Discrepancy.sum(),'households with contradicting targets, out of 2988 households in the train dataset')

**Lets see the data for 85 households that have discrepancy in target poverty level**

In [ ]:
Discrepancy_Index=(train.groupby('idhogar')['Target'].transform('nunique')>1)
HHID_Discrepancy=train.loc[Discrepancy_Index,'idhogar'].unique()
#household with contradicting target
train.loc[train['idhogar'].isin(HHID_Discrepancy),['idhogar','parentesco1','Target']].head()

**Judging from the data, the household head target might not be necessary true. Although prediction scoring is based on household head target, we should be able to safely replace the household target using the mode target of the household.**


In [ ]:

for HH in HHID_Discrepancy:
    Targets= (train.loc[train['idhogar']==HH,'Target'])

    if Targets.mode().shape[0] >1:
        for i in Targets.index:
            if train.loc[i,'parentesco1']==1:
                HeadTarget= train.loc[i,"Target"]    
        for i in Targets.index:
            train.loc[i,'Target']=HeadTarget
    elif Targets.mode().shape[0]==1:
        for i in Targets.index:
            TrueTarget=int(Targets.mode())
            train.loc[i,'Target']=TrueTarget
        
# Check for household targets discrepancy again for confirmation
target_Discrepancy=(train.groupby('idhogar')['Target'].nunique()>1)

print('There are ',target_Discrepancy.sum(),'households with contradicting targets, out of 2988 households in the train dataset')

train.head()
train.shape

**There are 9 columns where the attributes are the squared of other attributes. We do not need those in our model as the model are smart enough to detect non-linear relationship. **

**Remove (SQBescolari, SQBage, SQBHogar_ttal,SQBedjefe, SQBhogar_nin,SQBovercrowding, SQBdependency, SQBMeaned, agesq)**

In [ ]:
train=train.drop(columns=train.columns[133:142],axis=1)

test=test.drop(columns=test.columns[133:142],axis=1)

**Normalize int and float type columns for boxplot**

In [ ]:
min_max_scaler=preprocessing.MinMaxScaler()
trainNorm= train.select_dtypes(include='int64')
trainNorm=pd.DataFrame(min_max_scaler.fit_transform(trainNorm))
trainNorm.columns=train.select_dtypes(include='int64').columns


testNorm= test.select_dtypes(include='int64')
testNorm=pd.DataFrame(min_max_scaler.fit_transform(testNorm))
testNorm.columns=test.select_dtypes(include='int64').columns

In [ ]:
train_boxplot = trainNorm.iloc[:,2:134].boxplot(figsize=(140,5))
#Click on figure to expand

In [ ]:
test_boxplot = testNorm.iloc[:,2:134].boxplot(figsize=(140,5))
#Click on figure to expand

**Since prediction is scored only for the household head. We need to make new features that is household level and not individual. **

In [ ]:
#Setting new features for household specific in train data

#Number of Adults not including seniors >65
train['Adults']=train['hogar_adul']-train['hogar_mayor']
#Number of children < 19yo and seniors>65
train['Dependents']=train['hogar_nin']+train['hogar_mayor']
#Number of teenager from 12 to 19
train['Teenagers']=train['hogar_nin']-train['r4t1']
#Dependency is number of dependents per adults. This replaces the original dependency data from dataset.
train['dependency']=train['Dependents']/train['Adults']
#Percentage of Adults in household
train['P_Adults']=train['Adults']/train['hogar_total']
#Percentage of Male Adults in household
train['P_Adults_Male']=train['r4h3']/train['hogar_total']
#Percentage Female Adults in household
train['P_Adults_Female']=train['r4m3']/train['hogar_total']
#Percentage Children <19yo in household
train['P_Children']=train['hogar_nin']/train['hogar_total']
#Percentage of Seniors in household
train['P_Seniors']=train['hogar_mayor']/train['hogar_total']
#Percentage of Teenagers in household
train['P_Teenagers']=train['Teenagers']/train['hogar_total']
#Rent per person in household)
train['RentHH']=train['v2a1']/train['hogar_total']
#Rent per Adult in household
train['RentAdults']=train['v2a1']/train['Adults']
#Tablet per person in household
train['Tablet_PP']=train['v18q1']/train['hogar_total']
#Mobile Phone per person in household
train['Phone_PP']=train['qmobilephone']/train['hogar_total']
#Bedroom per person in household
train['Bedroom_PP']=train['bedrooms']/train['hogar_total']
#Appliance scoring. Higher the better
train['Appliances']=train['refrig']+train['computer']+train['television']
#Household size Difference
train['HHS_Diff']=train['tamviv']-train['hhsize']


#New Scoring For Education Level
for i in train.index:
    if train.loc[i,"instlevel9"] ==1 :
        train.loc[i,'EduLevel']= 6
    elif train.loc[i,"instlevel8"] ==1 :
        train.loc[i,'EduLevel']= 5
                   #higher scoring for completing tertiary education
    elif train.loc[i,"instlevel7"] ==1 :
        train.loc[i,'EduLevel']= 3                   
    elif train.loc[i,"instlevel5"]==1 :
        train.loc[i,'EduLevel']= 2
    elif train.loc[i,["instlevel4","instlevel3","instlevel6"]].sum() >0 :
        train.loc[i,'EduLevel']= 1                   
    else:
        train.loc[i,'EduLevel']= 0
    

train.head()

#We replicate the same for test data since we need the same features for prediction

test['Adults']=test['hogar_adul']-test['hogar_mayor']
test['Dependents']=test['hogar_nin']+test['hogar_mayor']
test['Teenagers']=test['hogar_nin']-test['r4t1']
test['dependency']=test['Dependents']/test['Adults']
test['P_Adults']=test['Adults']/test['hogar_total']
test['P_Adults_Male']=test['r4h3']/test['hogar_total']
test['P_Adults_Female']=test['r4m3']/test['hogar_total']
test['P_Children']=test['hogar_nin']/test['hogar_total']
test['P_Seniors']=test['hogar_mayor']/test['hogar_total']
test['P_Adultish']=test['Teenagers']/test['hogar_total']
test['RentHH']=test['v2a1']/test['hogar_total']
test['RentAdults']=test['v2a1']/test['Adults']
test['Tablet_PP']=test['v18q1']/test['hogar_total']
test['Phone_PP']=test['qmobilephone']/test['hogar_total']
test['Bedroom_PP']=test['bedrooms']/test['hogar_total']
test['Appliances']=test['refrig']+test['computer']+test['television']
test['HHS_Diff']=test['tamviv']-test['hhsize']

#New Scoring For Education Level
for i in test.index:
    if test.loc[i,"instlevel9"] ==1 :
        test.loc[i,'EduLevel']= 6
    elif test.loc[i,"instlevel8"] ==1 :
        test.loc[i,'EduLevel']= 5
                   #higher scoring for completing tertiary education
    elif test.loc[i,"instlevel7"] ==1 :
        test.loc[i,'EduLevel']= 3                   
    elif test.loc[i,"instlevel5"]==1 :
        test.loc[i,'EduLevel']= 2
    elif test.loc[i,["instlevel4","instlevel3","instlevel6"]].sum() >0 :
        test.loc[i,'EduLevel']= 1                   
    else:
        test.loc[i,'EduLevel']= 0

test.head()

**Now we want to aggregate existing features to be representable in household level**


In [ ]:
List_Mean = ['rez_esc', 'male', 'female', 'estadocivil1', 'estadocivil2', 'estadocivil3', 'estadocivil4', 'estadocivil5',
             'estadocivil6', 'estadocivil7', 'parentesco2','parentesco3', 'parentesco4', 'parentesco5', 'parentesco6', 'parentesco7',
             'parentesco8', 'parentesco9', 'parentesco10', 'parentesco11', 'parentesco12','instlevel1', 'instlevel2', 'instlevel3',
             'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9','overcrowding']

List_Summary = ['age', 'escolari','dis','EduLevel']


trainGP = pd.DataFrame()
testGP = pd.DataFrame()

for item in List_Mean:
    group_train_mean = train[item].groupby(train['idhogar']).mean()
    group_test_mean = test[item].groupby(test['idhogar']).mean()
    new_col = item + '_mean'
    trainGP[new_col] = group_train_mean
    testGP[new_col] = group_test_mean

for item in List_Summary:
    for function in ['mean','std','min','max','sum']:
        group_train = train[item].groupby(train['idhogar']).agg(function)
        group_test = test[item].groupby(test['idhogar']).agg(function)
        new_col = item + '_' + function
        trainGP[new_col] = group_train
        testGP[new_col] = group_test
        
#adding one final feature
        
trainGP['age_extreme']=trainGP['age_max']-trainGP['age_min']
testGP['age_extreme']=testGP['age_max']-testGP['age_min']
trainGP['escolari_extreme']=trainGP['escolari_max']-trainGP['escolari_min']
testGP['escolari_extreme']=testGP['escolari_max']-testGP['escolari_min']
        
trainGP.head()
testGP.head()

Now we want to merge the new feature together with the aggregated features of household

In [ ]:
trainGP = trainGP.reset_index()
testGP = testGP.reset_index()

trainML = pd.merge(train, trainGP, on='idhogar')
testML = pd.merge(test, testGP, on='idhogar')

#fill all na as 0
trainML.fillna(value=0, inplace=True)
testML.fillna(value=0, inplace=True)

trainML.head()
testML.head()

In [ ]:
submission = testML[['Id']]
#remove features that are not relevant to reduce size
trainML.drop(columns=['idhogar','Id','tamhog','r4t3','hhsize','hogar_adul','edjefe','edjefa'],inplace=True)
testML.drop(columns=['idhogar','Id','tamhog','r4t3','hhsize','hogar_adul','edjefe','edjefa'],inplace=True)

correlation=trainML.corr()
correlation = correlation ['Target'].sort_values(ascending=False)
print(f'The most 20 positive feature: \n{correlation.head(20)}')
print('*'*50)

print(f'The most 20 negative feature: \n{correlation.tail(20)}')

With the train and test data finally preprocessed. We will make use of the Light GBM model and parameters adjusted from MIsha Lisovyi.https://www.kaggle.com/mlisovyi/lighgbm-hyperoptimisation-with-f1-macro

In [ ]:
y= trainML['Target']
trainML.drop(columns=['Target'], inplace=True)


In [ ]:
#parameter value is copied
clf = lgb.LGBMClassifier(max_depth=8, learning_rate=0.1, objective='multiclass',
                             random_state=None, silent=True, metric='multi_logloss', 
                             n_jobs=4, n_estimators=5000, class_weight='balanced',
                             colsample_bytree =  0.93, min_child_samples = 95, num_leaves = 80, subsample = 0.96)


clf

In [ ]:
kfold = 5
kf = StratifiedKFold(n_splits=kfold, shuffle=True)


In [ ]:
#QuickCheck to makesure train and test have have same columns size
trainML.shape
testML.shape

In [ ]:
predicts_result = []
for trainML_index, testML_index in kf.split(trainML, y):
    print("###")
    X_train, X_val = trainML.iloc[trainML_index], trainML.iloc[testML_index]
    y_train, y_val = y.iloc[trainML_index], y.iloc[testML_index]
    clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
            early_stopping_rounds=400, verbose=100)
    predicts_result.append(clf.predict(testML))

In [ ]:
indices = np.argsort(clf.feature_importances_)[::-1]
indices = indices[:75]

# Visualise these with a barplot
plt.subplots(figsize=(20, 15))
g = sns.barplot(y=trainML.columns[indices], x = clf.feature_importances_[indices], orient='h')
g.set_xlabel("Relative importance",fontsize=12)
g.set_ylabel("Features",fontsize=12)
g.tick_params(labelsize=9)
g.set_title("LightGBM feature importance");

In [ ]:
submission['Target'] = np.array(predicts_result).mean(axis=0).round().astype(int)
submission.head()

In [ ]:

submission.to_csv('SubmissionV5_15May2019.csv',index=False)
